In [7]:
import pandas as pd

duda=pd.read_excel('Duda_MSSNG.xlsx',sheet_name="Table S4", skiprows=1)

In [8]:
gmt_list=list(set(duda["GENE_ID"].tolist()))
syb_list=list(set(duda["GENE_SYMBOL"].tolist()))
len(gmt_list)

214

In [9]:

from gprofiler import GProfiler

gp = GProfiler(return_dataframe=True)
gp_convert_df =gp.convert(organism  = 'hsapiens',
                                query= gmt_list,
                                    target_namespace= 'ENSG',
                                    numeric_namespace='ENTREZGENE_ACC',)
gp_convert_df = gp_convert_df[['incoming', 'converted']]

ensemble_list = gp_convert_df['converted'].tolist()
gp_convert_df

,incoming,converted
0,140803,ENSG00000119121
1,6660,ENSG00000134532
2,202243,ENSG00000183323
3,10765,ENSG00000117139
4,91662,ENSG00000142405
...,...,...
209,57835,ENSG00000188687
210,4076,ENSG00000135387
211,3061,ENSG00000121764
212,55802,ENSG00000272886


In [10]:
gene_string = '\t'.join(ensemble_list)
print(gene_string)
gmt_file= f'MSSNG_LOF\tMutation: LOF\t' + gene_string
print(gmt_file)

ENSG00000119121	ENSG00000134532	ENSG00000183323	ENSG00000117139	ENSG00000142405	ENSG00000136158	ENSG00000137073	ENSG00000105722	ENSG00000183762	ENSG00000150938	ENSG00000131969	ENSG00000166148	ENSG00000029363	ENSG00000242265	ENSG00000148795	ENSG00000160862	ENSG00000100567	ENSG00000107731	ENSG00000116539	ENSG00000175216	ENSG00000089327	ENSG00000174032	ENSG00000198830	ENSG00000247596	ENSG00000085644	ENSG00000196865	ENSG00000160741	ENSG00000144504	ENSG00000102781	ENSG00000221909	ENSG00000189241	ENSG00000023287	ENSG00000164309	ENSG00000080603	ENSG00000100288	ENSG00000188419	ENSG00000142235	ENSG00000157741	ENSG00000164796	ENSG00000171862	ENSG00000196367	ENSG00000100815	ENSG00000169057	ENSG00000082781	None	ENSG00000079432	ENSG00000137204	ENSG00000086288	ENSG00000130940	ENSG00000215301	ENSG00000169718	ENSG00000157510	ENSG00000095787	ENSG00000170860	ENSG00000124214	ENSG00000149930	ENSG00000197299	ENSG00000112039	ENSG00000166436	ENSG00000176273	ENSG00000162825	ENSG00000137857	ENSG00000011201	ENS

In [11]:
with open('duda.gmt', 'w') as f:
    f.write(gmt_file + "\n")      
